In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('symptoms_diseases_train.csv')

In [3]:
symptoms=dataset['query']
label=dataset['response']

In [4]:
train_df = pd.DataFrame({
    "Symptoms": symptoms,
    "Label": label
})

In [5]:
label_counts = train_df['Label'].value_counts()

valid_labels = label_counts[label_counts >= 400].index

filtered_train_df = train_df[train_df['Label'].isin(valid_labels)]



In [6]:
train_df = filtered_train_df.groupby('Label').apply(lambda x: x.head(400))
test_df= filtered_train_df.groupby('Label').apply(lambda x: x[400:410])


In [ ]:
def remove_phrase(symptoms, phrase):

  new_symptoms = symptoms.replace(phrase, "")
  return new_symptoms

updated_symptoms = remove_phrase(train_df['Symptoms'], "Having these specific symptoms :->")

train_df["Symptoms"] = train_df["Symptoms"].apply(lambda x: remove_phrase(x, "Having these specific symptoms :->"))

train_df.head()

In [8]:

train_df = train_df.sample(frac=1).reset_index(drop=True)


In [34]:
labels=train_df['Label']
Symptoms=train_df['Symptoms']



In [ ]:
len(Symptoms)

In [ ]:
labels.head()

In [11]:
Symptoms[150000]

'lower abdominal pain ,intermenstrual bleeding ,pelvic pain ,burning abdominal pain ,problems during pregnancy ,spotting or bleeding during pregnancy ,blood clots during menstrual periods '

In [ ]:
labels[150000]

In [37]:
class_names=set(labels)


In [ ]:
unique_labels = set(labels)
unique_label_count = len(unique_labels)
print(f"Number of unique labels: {unique_label_count}")


In [ ]:
unique_labels

In [17]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(labels)

num_classes = len(label_encoder.classes_)
train_labels_onehot = tf.one_hot(train_labels_encoded, depth=unique_label_count)

In [ ]:
train_labels_onehot[0]

In [ ]:
num_classes

In [ ]:
max_words = max([len(symptom.split()) for symptom in Symptoms])

unique_words = set()

for symptom in Symptoms:
  for word in symptom.split():
    unique_words.add(word)

num_unique_words = len(unique_words)

num_unique_words,max_words


In [ ]:
len(train_labels_onehot)

In [43]:
from sklearn.model_selection import train_test_split
import numpy as np

Symptoms = np.array(Symptoms)
train_labels_onehot = np.array(train_labels_onehot)

X_train, X_test, y_train, y_test = train_test_split(Symptoms, train_labels_onehot, test_size=0.2, shuffle=True, random_state=42)


In [46]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(128).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.shuffle(buffer_size=1024).batch(128).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [45]:
from tensorflow.keras.layers import TextVectorization , Embedding

text_vectorizer = TextVectorization(max_tokens=num_unique_words, pad_to_max_tokens=True)
text_vectorizer.adapt(Symptoms)

embedded_symptoms=Embedding(input_dim=num_unique_words,output_dim=max_words,mask_zero=True,)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers


inputs = tf.keras.Input(shape=(1,), dtype='string')

x1 = text_vectorizer(inputs)
x1 = embedded_symptoms(x1)
x1 = layers.GlobalAveragePooling1D()(x1)
x1 = layers.Dense(128, activation='relu')(x1)
x1 = layers.Dropout(0.5)(x1)
x1 = layers.Dense(64, activation='relu')(x1)
outputs1 = layers.Dense(num_classes, activation='softmax')(x1)
model1 = tf.keras.Model(inputs, outputs1)
model1.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
history1 = model1.fit(train_dataset, epochs=15, validation_data=test_dataset)
